In [1]:
import pygame
import numpy as np

# Define constants
WIDTH, HEIGHT = 800, 800
GRID_SIZE_X, GRID_SIZE_Y = 80, 80
CELL_SIZE = WIDTH // GRID_SIZE_X
FPS = 60
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
ORANGE = (255, 165, 0)
GREEN = (0, 128, 0)

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()

# Create grids
field_pheromone_home = np.zeros((GRID_SIZE_X, GRID_SIZE_Y)).astype(float)
field_pheromone_search = np.zeros((GRID_SIZE_X, GRID_SIZE_Y)).astype(float)
field_food = np.zeros((GRID_SIZE_X, GRID_SIZE_Y)).astype(float)
field_food[3:20, 20:60] = 3
field_nest = np.zeros((GRID_SIZE_X, GRID_SIZE_Y)).astype(float)
field_nest[35:46, 35:46] = 1

directions = [[0, 1], [-1, 1], [-1, 0], [-1, -1], [0, -1], [1, -1], [1, 0], [1, 1]]


class Ant:
    def __init__(self, position, direction):
        self.position = np.array(position)
        self.direction = direction
        self.has_food = False
        self.pheromone_strength = 1

    def search_step(self):
        field_pheromone_search[tuple(self.position)] += self.pheromone_strength
        self.update_pheromone(field_pheromone_home)
        self.pheromone_strength *= 0.5

    def home_step(self):
        field_pheromone_home[tuple(self.position)] += self.pheromone_strength
        self.update_pheromone(field_pheromone_search)
        self.pheromone_strength *= 0.9

    def update_pheromone(self, pheromone_field):
        grid_size_x, grid_size_y = pheromone_field.shape
        pheromones_in_direction = [
            pheromone_field[tuple(np.remainder(self.position + directions[i_dir], [grid_size_x, grid_size_y]))]
            for i_dir in create_range(self.direction - 1, (self.direction + 2) % 8, 8)
        ]
        probabilities_in_directions = [(2 + i) ** 9 for i in pheromones_in_direction]
        probabilities_in_directions /= np.sum(probabilities_in_directions)
        chosen_direction = np.random.choice(
            [(self.direction - 1) % 8, self.direction, (self.direction + 1) % 8], p=probabilities_in_directions
        )
        self.direction = chosen_direction
        self.position = np.remainder(self.position + directions[chosen_direction], [grid_size_x, grid_size_y])
        self.pheromone_strength *= 0.9

    def check_for_food(self):
        pointed_point = np.remainder(self.position + directions[self.direction], [GRID_SIZE_X, GRID_SIZE_Y])
        if_food_in_direction = bool(field_food[pointed_point[0], pointed_point[1]])
        if if_food_in_direction or bool(field_food[self.position[0], self.position[1]]):
            if self.has_food:
                self.direction += 4
                self.direction = self.direction % 8
            else:
                self.has_food = True

            field_food[pointed_point[0], pointed_point[1]] -= 1
            self.pheromone_strength = 1

    def check_for_nest(self):
        pointed_point = np.remainder(self.position + directions[self.direction], [grid_size_x, grid_size_y])
        if_nest_in_direction = bool(field_nest[pointed_point[0], pointed_point[1]])
        if if_nest_in_direction or bool(field_nest[self.position[0], self.position[1]]):
            if self.has_food:
                self.has_food = False
                self.pheromone_strength = 1


def create_range(start, stop, modulo):
    result = []
    index = start
    while index != stop:
        result.append(index)
        index = (index + 1) % modulo
    return result


ants_list = [Ant([GRID_SIZE_X // 2, GRID_SIZE_Y // 2], np.random.choice(np.arange(8))) for _ in range(150)]

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE):
            running = False

    screen.fill(WHITE)

    for ant in ants_list:
        ant.check_for_food()
        if ant.has_food:
            ant.home_step()
        else:
            ant.search_step()

    field_pheromone_search *= 0.99
    field_pheromone_home *= 0.99
    field_food[field_food < 0] = 0
    field_pheromone_search[tuple([GRID_SIZE_X // 2, GRID_SIZE_Y // 2])] = 0

    # Draw ants on the screen
    for ant in ants_list:
        x, y = ant.position
        color = BLUE if ant.has_food else ORANGE
        pygame.draw.circle(screen, color, (x * CELL_SIZE + CELL_SIZE // 2, y * CELL_SIZE + CELL_SIZE // 2),
                           CELL_SIZE // 3)

    for x in range(GRID_SIZE_X):
        for y in range(GRID_SIZE_Y):
            rect = pygame.Rect(x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE)
            if field_food[x, y]:
                screen.fill(BLUE, rect)
            if field_pheromone_search[x, y]:
                color = tuple(int(min(field_pheromone_search[x, y] * 5, 255)) for _ in range(3))
                screen.fill(ORANGE, rect)
            if field_pheromone_home[x, y]:
                color = tuple(int(min(field_pheromone_home[x, y] * 5, 255)) for _ in range(3))
                screen.fill(GREEN, rect)

    pygame.display.flip()
    clock.tick(FPS)

pygame.quit()


pygame 2.1.2 (SDL 2.0.20, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html
